In [34]:
import numpy as np
import pandas as pd
from supplemental_english import GOVERNMENT_CODES
from supplemental_english import REGION_CODES

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier,XGBRegressor

In [50]:
df = pd.read_csv('data/final_train.csv')
test_df = pd.read_csv('data/final_test.csv')
test_id_df = pd.read_csv('data/test.csv')

In [38]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['price']),df['price'],test_size = 0.2, random_state=2)

In [39]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41308 entries, 23478 to 23720
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            41308 non-null  int64
 1   plate         41308 non-null  int64
 2   date          41308 non-null  int64
 3   forbidden     41308 non-null  int64
 4   advantage     41308 non-null  int64
 5   significance  41308 non-null  int64
 6   city          41308 non-null  int64
dtypes: int64(7)
memory usage: 2.5 MB


In [40]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7695 entries, 0 to 7694
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            7695 non-null   int64
 1   plate         7695 non-null   int64
 2   date          7695 non-null   int64
 3   forbidden     7695 non-null   int64
 4   advantage     7695 non-null   int64
 5   significance  7695 non-null   int64
 6   city          7695 non-null   int64
dtypes: int64(7)
memory usage: 420.9 KB


In [41]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBRegressor()

In [42]:
clfs = {
    'SVC' : svc,#
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, #
    'RF': rfc, #
    'AdaBoost': abc, 
    'BgC': bc, #
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [43]:
# score calculator
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))


In [44]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    smape_score = smape(y_test, y_pred)
    
    return smape_score

In [45]:
for name,clf in clfs.items():
    
    smape_score = train_classifier(clf, X_train,y_train,X_test,y_test)
    
    print("For ",name)
    print("smape - ",smape_score)

For  SVC
smape -  80.92433043427242
For  KN
smape -  91.43535568317074
For  NB
smape -  125.09708596352837
For  DT
smape -  83.33363245424817
For  LR
smape -  83.35867191069286
For  RF
smape -  76.20799950923164
For  AdaBoost
smape -  81.99931095994494
For  BgC
smape -  77.38190098681883
For  ETC
smape -  76.83021282097559
For  GBDT
smape -  116.15569017874515
For  xgb
smape -  87.94426837668226


In [48]:
rfc.fit(X_train,y_train)
y_pred = rfc.predict(test_df)

In [51]:
pred=pd.DataFrame(y_pred)
datasets=pd.concat([test_id_df['id'],pred],axis=1)
datasets.columns=['id','price']
datasets.to_csv('outputs/xgb_sample_submission.csv',index=False)